In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import LSTM,Dense, Concatenate,Bidirectional,Input,LSTMCell
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy,CosineSimilarity
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import AdditiveAttention
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import tensorflow.keras.backend as K
from tensorflow_addons.seq2seq import BeamSearchDecoder



In [3]:
#importing vocabulary
ds_path='/home/al/Desktop/nnfl/word2vecmod.txt'
data = np.loadtxt(ds_path,dtype=str,usecols=(0))
print(data)
print(data.shape[0])

['a' 'abandon' 'ability' ... '<start>' '<end>' '<pad>']
5035


In [4]:
#loading word vectors for getting unkown words
tmp_file = get_tmpfile(datapath("/home/al/Desktop/nnfl/new"))
word_model = KeyedVectors.load(tmp_file)

In [5]:
#list of words in word vectors
data2= np.loadtxt('/home/al/Desktop/nnflproj/glove.6B.300d.txt',dtype=str,usecols=(0))

In [6]:
#making word-to-index and index-to-word dictionaries
word_to_ix=dict(zip(data,range(0,data.shape[0])))
ix_to_word=dict(zip(range(0,data.shape[0]),data))

In [15]:
#constants
voc=data.shape[0]#for <start>,<end>,<pad>
ip_len=15
op_len=15
n_s=64+op_len#no of decoder lstm
n_a=64#no 
batch_size=20


In [16]:
#one-hot function
def oh(Xq,ip_ln):
    Xoh=np.zeros([Xq.shape[0],ip_ln,voc])
    for i in range(Xq.shape[0]):
        for j in range(ip_ln):
            Xoh[i,j,int(Xq[i,j])]=1
    return Xoh

In [20]:
def ev(s1):
        s1='<start> '+s1+' <end>'
        Xt=np.zeros(shape=[ip_len])
        #ds.shape[0]-(ds.shape[0]%1000)+1)
        w1=np.array(s1.lower().split())
        for i in range(0,ip_len):    
            w=w1[i]
            if i==ip_len-1:
                #w1_sum/=50
                Xt[i]=voc-2#for<end>
                
                break
            if w=='<start>':
                Xt[i]=voc-3
                continue
            if w=='<end>':
                Xt[i]=voc-2#<end>
                i+=1
                while i<ip_len:
                    Xt[i]=voc-1#padding
                    i+=1
                break
            if w in data:                
                Xt[i]=word_to_ix[w]
            else:
                try:
                    temp=word_model.most_similar(positive=[w],negative=[],restrict_vocab=4095)#for<unk>
                    
                    Xt[i]=word_to_ix[temp[0][0]]
                except KeyError:
                    Xt[i]=word_to_ix['it']
                    
            return Xt.reshape([1,ip_len])


In [24]:
dull_oh=np.ndarray(shape=[4,15,5035])

dull_sen=['i do not know','ok','i have no idea','maybe']
i=0
for el in dull_sen:
  a=ev(el)
  a=oh(a,ip_len)
  dull_oh[i]=a
  i+=1

[[[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [65]:
#model
enc_ip=Input(shape=(ip_len,voc))
enc_lstm1=LSTM(100,return_sequences=True)
enc_lstm2=LSTM(100,return_sequences=True)
enc_lstm3=LSTM(100,return_sequences=True)
enc_lstm4=LSTM(1000,return_state=True,return_sequences=True)
en=enc_lstm1(enc_ip)
en=enc_lstm2(en)
en=enc_lstm3(en)
enc_op,h,c=enc_lstm4(en)
concat=Concatenate(axis=-1)
enc_states=[h,c]
dec_ip=Input(shape=(op_len,voc))
dense=Dense(voc,activation='softmax')
dec_lstm=LSTM(1000,return_sequences=True,return_state=True)
dec_op,_,_=dec_lstm(dec_ip,initial_state=enc_states)

op=dense(dec_op)
model=Model([enc_ip,dec_ip],enc_op)

model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 15, 5035)]   0                                            
__________________________________________________________________________________________________
lstm_29 (LSTM)                  (None, 15, 100)      2054400     input_15[0][0]                   
__________________________________________________________________________________________________
lstm_30 (LSTM)                  (None, 15, 100)      80400       lstm_29[0][0]                    
__________________________________________________________________________________________________
lstm_31 (LSTM)                  (None, 15, 100)      80400       lstm_30[0][0]                    
___________________________________________________________________________________________

In [66]:
enc_model=Model(enc_ip,enc_op)
enc_model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])
print(enc_model.summary())

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 15, 5035)]        0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 15, 100)           2054400   
_________________________________________________________________
lstm_30 (LSTM)               (None, 15, 100)           80400     
_________________________________________________________________
lstm_31 (LSTM)               (None, 15, 100)           80400     
_________________________________________________________________
lstm_32 (LSTM)               [(None, 15, 1000), (None, 4404000   
Total params: 6,619,200
Trainable params: 6,619,200
Non-trainable params: 0
_________________________________________________________________
None


In [124]:
#model
c=enc_model.predict(oh(ev('hi'),ip_len))
a=tf.transpose(c,perm=[1,0,2])
seq_len=[ip_len]
paths=3
decop,log_prob=tf.nn.ctc_beam_search_decoder(a,seq_len,beam_width=5,top_paths=paths)
d=[]
for i in range(paths):
      d+=[decop[i].values]
  

[[[ 8.6683085e-06  1.0323745e-05  1.3909574e-05 ...  2.8876404e-06
    7.9924148e-06  3.5765036e-06]
  [ 2.1260574e-05  3.4408182e-05  3.2315482e-05 ...  1.0514347e-05
    1.8351093e-05  5.0931635e-06]
  [ 2.3842460e-05  6.2993597e-05  4.8856687e-05 ...  2.3534401e-05
    3.1118110e-05  1.6299377e-06]
  ...
  [-4.3421233e-04 -8.9967623e-04 -3.3774049e-04 ...  3.9662290e-04
    6.2591641e-04 -1.4626371e-05]
  [-4.1793531e-04 -1.0709496e-03 -4.1476317e-04 ...  4.1842458e-04
    7.2325225e-04 -1.0395716e-05]
  [-3.8401029e-04 -1.2351057e-03 -4.9052044e-04 ...  4.3263086e-04
    8.2021009e-04 -8.5474794e-06]]]


In [120]:
print(d)

tf.Tensor([], shape=(0,), dtype=int64)


In [126]:
c=oh(ev('hi'),ip_len)
a2=tf.transpose(c,perm=[1,0,2])
decop,log_pro=tf.nn.ctc_beam_search_decoder(a2,seq_len,beam_width=5,top_paths=paths)
d2=[]
for i in range(paths):
    d2+=[decop[i].values]

In [127]:
print(d2)

[<tf.Tensor: shape=(3,), dtype=int64, numpy=array([5032, 1258,    0])>, <tf.Tensor: shape=(4,), dtype=int64, numpy=array([5032, 1258,    0,    3])>, <tf.Tensor: shape=(4,), dtype=int64, numpy=array([5032, 1258,    0,    2])>]


In [113]:
print(log_pro[0])

NameError: name 'log_pro' is not defined

In [0]:
#creating checkpoint
filepath="/home/al/Desktop/nnfl final/weightsfinv100.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_accuracy',verbose=1, save_best_only=True, mode='max')
#model.load_weights(filepath)

In [69]:
d_path='/home/al/Desktop/nnfl final/nds-a'
idx=0
idx2=0
p=3
for e  in 'a':
#bcdefghi':

    for f  in 'abc':
        #defghijklmnopqrstuvwxyz':
       
        ds = np.loadtxt(d_path+e+f+'.csv',delimiter='\n',dtype='str')
        num_sen=ds.shape[0]
        idx=0
        Xtr=np.ndarray(shape=[num_sen,ip_len])
        for j in range(0,num_sen):
                #ds.shape[0]-(ds.shape[0]%1000)+1)
                w1=np.array(ds[j].lower().split())
                for i in range(0,ip_len):    
                    w=w1[i]
                    if i==ip_len-1:
                        #w1_sum/=50
                        Xtr[j,i]=voc-2#for<end>
                        
                        break
                    if w=='<start>':
                        Xtr[j,i]=voc-3
                        continue
                    if w=='<end>':
                        Xtr[j,i]=voc-2#<end>
                        i+=1
                        while i<ip_len:
                            Xtr[j,i]=voc-1#padding
                            i+=1
                        break
                    if w in data:                
                        Xtr[j,i]=word_to_ix[w]
                    else:
                        try:
                            temp=word_model.most_similar(positive=[w],negative=[],restrict_vocab=4095)#for<unk>
                            idx2+=1
                            Xtr[j,i]=word_to_ix[temp[0][0]]
                        except KeyError:
                            Xtr[j,i]=word_to_ix['it']
                            idx+=1


                        
                           
                if j%1000==0:
                    print(j)
    print(idx)
    print(idx2)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000


In [0]:
#train

for kl in range(100,num_sen,100):
    u=oh(Xtr[kl-100:kl+1],ip_len)
    u=tf.convert_to_tensor(u)
    model.fit([u[0:-1],u[0:-1]],u[1:,:,:],epochs=1,batch_size=20,use_multiprocessing=True,callbacks=[checkpoint],verbose=2,validation_split=0.2,shuffle=True)
print('\n\n\n\n\ndone')

 20 samples

Epoch 00001: val_accuracy did not improve from 0.76333
80/80 - 0s - loss: 3.0884 - accuracy: 0.5383 - val_loss: 2.6375 - val_accuracy: 0.5733
Train on 80 samples, validate on 20 samples

Epoch 00001: val_accuracy did not improve from 0.76333
80/80 - 0s - loss: 3.2532 - accuracy: 0.5258 - val_loss: 2.7716 - val_accuracy: 0.5800
Train on 80 samples, validate on 20 samples

Epoch 00001: val_accuracy did not improve from 0.76333
80/80 - 0s - loss: 2.8199 - accuracy: 0.5550 - val_loss: 2.3967 - val_accuracy: 0.6367
Train on 80 samples, validate on 20 samples

Epoch 00001: val_accuracy did not improve from 0.76333
80/80 - 1s - loss: 2.7824 - accuracy: 0.5808 - val_loss: 2.5187 - val_accuracy: 0.6333
Train on 80 samples, validate on 20 samples

Epoch 00001: val_accuracy did not improve from 0.76333
80/80 - 0s - loss: 3.4502 - accuracy: 0.5042 - val_loss: 2.3640 - val_accuracy: 0.6267
Train on 80 samples, validate on 20 samples

Epoch 00001: val_accuracy did not improve from 0.763

KeyboardInterrupt: 